In [6]:
import os

mycwd = os.getcwd()

os.chdir("..")

In [5]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import logging
import matplotlib.pyplot as plt

tf.get_logger().setLevel(logging.ERROR)
from notebooks.load_dataset.dataset import classes, x_train, y_train, x_test, y_test

In [4]:
# Create classes
class_names = {
    0: classes[0],
    1: classes[1],
    2: classes[2],
    3: classes[3],
    4: classes[4],
    5: classes[5],
    6: classes[6],
    7: classes[7],
    8: classes[8],
    9: classes[9],
}

In [4]:
def preprocess_image_input(input_images):
    input_images = input_images.astype("float32")
    output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
    return output_ims


def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.resnet.ResNet50(
        input_shape=(224, 224, 3), include_top=False, weights="imagenet"
    )(inputs)
    return feature_extractor


def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x


def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7, 7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output


def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))

    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=classification_output)

    model.compile(
        optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [5]:
train_X = preprocess_image_input(x_train)
valid_X = preprocess_image_input(x_test)

In [9]:
model = define_compile_model()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

In [9]:
history = model.fit(
    train_X,
    y_train,
    epochs=5,
    validation_split=0.2,
    batch_size=64,
)

Epoch 1/3
  6/625 [..............................] - ETA: 5:43:33 - loss: 2.3384 - accuracy: 0.1432

: 

In [ ]:
model.save("resnet_best.keras")

In [6]:
model = tf.keras.models.load_model("../models/resnet_best.h5")

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

In [ ]:
loss, accuracy = model.evaluate(valid_X, y_test, batch_size=64)

In [2]:
# Y_test_list = Y_test.tolist()

# Y_test_named = [[class_names[label[0]]] for label in Y_test_list]

# Y_test_named = np.array(Y_test_named)

In [1]:
# from PIL import Image

# example_no = 3857

# sample = X_test[example_no].reshape((1, 32, 32, 3))
# predict_x = model.predict(sample)

# list_x = predict_x.tolist()
# sorted_x = sorted(list_x)
# pred_num = list_x[0].index(max(list_x[0]))
# pred_name = class_names[pred_num]

# image = Image.fromarray(X_test[example_no])
# img = image.resize((224, 224), Image.BILINEAR)
# plt.imshow(img)

# print(pred_name)